## かんたんなモデルで動作確認

In [ ]:
from pathlib import Path

import tensorflow as tf

import stock

In [ ]:
dataset_params = stock.dl.dataset.DatasetParams(
    symbols_csv_path=Path("~/work/Projects/stock/data/sp500_companies.csv"),
    data_dir=stock.DATA_DIR / "sp500",
    dataset_path=stock.DATA_DIR / "dataset" / "sp500_20221203.npy",
    input_width=30,
    shift=30,
    output_width=1,
)
dataset = stock.dl.dataset.Dataset(dataset_params)
ds = dataset.make_dataset(is_train=True)

In [ ]:
for x in ds.take(1):
    print(x[0].shape, x[1].shape)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(dataset.data.shape[1])
])
model.compile(loss=tf.keras.losses.MeanSquaredError(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=[tf.keras.metrics.MeanAbsoluteError()])
model.build(input_shape=[dataset.params.batch_size, dataset.params.input_width, dataset.data.shape[1]])
model.summary()

In [ ]:
history = model.fit(ds.as_numpy_iterator(), epochs=10)